In [1]:
!pip install fastapi uvicorn pyngrok nest_asyncio cloudinary moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import requests
import uuid
import moviepy.editor as mp
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import cloudinary
from cloudinary import uploader
import nest_asyncio
import uvicorn
from threading import Thread

# === Setup ===
SAVE_DIR = "./clips"
os.makedirs(SAVE_DIR, exist_ok=True)

# === Cloudinary Config (Update your credentials) ===
cloudinary.config(
    cloud_name="dxjjpowf1",
    api_key="276811499447944",
    api_secret="Eeb9f4MTWcmsPCxXsujuv46nOXs",
    secure=True
)

# === FastAPI App ===
app = FastAPI()

# === Enable CORS ===
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# === Helpers ===
def download_file(url, save_path):
    try:
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"Download error: {e}")
        return False

def upload_to_cloudinary(file_path, resource_type):
    try:
        response = uploader.upload(
            file_path,
            resource_type=resource_type,
            chunk_size=6000000
        )
        return response.get("secure_url")
    except Exception as e:
        print(f"Upload error: {e}")
        return None

# === Main Processing ===
@app.get("/cut_video")
def cut_video(
    cloudinary_url: str = Query(...),
    start_time: float = Query(...),
    end_time: float = Query(...)
):
    unique_id = str(uuid.uuid4())[:8]
    local_video_path = f"{SAVE_DIR}/video_{unique_id}.mp4"
    clipped_video_path = f"{SAVE_DIR}/clip_{unique_id}.mp4"
    clipped_audio_path = f"{SAVE_DIR}/clip_{unique_id}.wav"

    # Download the input video
    if not download_file(cloudinary_url, local_video_path):
        return JSONResponse(status_code=500, content={"error": "Failed to download video"})

    try:
        # Clip the video
        video = mp.VideoFileClip(local_video_path)
        clip = video.subclip(start_time, min(end_time, video.duration))

        # Save clipped video
        clip.write_videofile(clipped_video_path, audio_codec='aac', verbose=False, logger=None)

        # Save audio to WAV
        clip.audio.write_audiofile(clipped_audio_path, verbose=False, logger=None)

        # Upload both
        video_url = upload_to_cloudinary(clipped_video_path, "video")
        wav_url = upload_to_cloudinary(clipped_audio_path, "raw")

        # Clean up
        clip.close()
        video.close()
        os.remove(local_video_path)
        os.remove(clipped_video_path)
        os.remove(clipped_audio_path)

        return {
            "status": "success",
            "video_url": video_url,
            "wav_url": wav_url
        }

    except Exception as e:
        return JSONResponse(status_code=500, content={"error": f"Processing failed: {str(e)}"})

# === Ngrok Tunnel ===
def run_ngrok():
    ngrok.set_auth_token("2vyGA7Tyh2LoFJeFRO17NUXS38Z_5qEBYN7jcEn4LTgH6cvk5")
    public_url = ngrok.connect(8000)
    print(f"\n🚀 Public URL: {public_url}\n")

def start():
    Thread(target=run_ngrok).start()

# === Main Entry ===
if __name__ == "__main__":
    start()
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)


  if event.key is 'enter':



INFO:     Started server process [317]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



🚀 Public URL: NgrokTunnel: "https://3dfa-34-10-17-74.ngrok-free.app" -> "http://localhost:8000"

INFO:     2405:acc0:1307:4e5d:4d3c:624d:70fb:364d:0 - "OPTIONS /cut_video?cloudinary_url=https:%2F%2Fres.cloudinary.com%2Fde1yfnzdz%2Fvideo%2Fupload%2Fv1745502872%2Fcnmemcg4obnphgcsnst4.mp4&start_time=0&end_time=8 HTTP/1.1" 200 OK
INFO:     2405:acc0:1307:4e5d:4d3c:624d:70fb:364d:0 - "GET /cut_video?cloudinary_url=https:%2F%2Fres.cloudinary.com%2Fde1yfnzdz%2Fvideo%2Fupload%2Fv1745502872%2Fcnmemcg4obnphgcsnst4.mp4&start_time=0&end_time=8 HTTP/1.1" 200 OK
INFO:     2405:acc0:1307:4e5d:4d3c:624d:70fb:364d:0 - "OPTIONS /cut_video?cloudinary_url=https:%2F%2Fres.cloudinary.com%2Fde1yfnzdz%2Fvideo%2Fupload%2Fv1745503722%2Fvupwz7zigz6axvtseugw.mp4&start_time=0&end_time=8 HTTP/1.1" 200 OK
INFO:     2405:acc0:1307:4e5d:4d3c:624d:70fb:364d:0 - "GET /cut_video?cloudinary_url=https:%2F%2Fres.cloudinary.com%2Fde1yfnzdz%2Fvideo%2Fupload%2Fv1745503722%2Fvupwz7zigz6axvtseugw.mp4&start_time=0&end_time=8 HTT